# Raptors Box Score WebScraping

In [674]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

## Grab 1 Box Score

In [777]:
# Grab URL
test_url_1= 'https://www.basketball-reference.com/boxscores/202311150TOR.html'

In [778]:
# extract data from test
data = requests.get(test_url)
# open and save data
with open('Box_Scores/202311150TOR.html', 'w+') as f:
    f.write(data.text)

In [779]:
# read data
with open('Box_Scores/202311150TOR.html') as f:
    page = f.read()

In [780]:
# # read html with bs4
# soup = BeautifulSoup(data, "html.parser")
# # read data
# with open('Box_Scores/202311150TOR.html') as f:
#     page = f.read()

In [781]:
# Remove unnecessary top row with name: th.over_header.center
soup.find('tr', class_= 'over_header').decompose()
# name of table class: box-TOR-game-basic, grab table
rap_box = soup.find_all(id='box-TOR-game-basic')

In [782]:
# turn table class into a dataframe
rap_table = pd.read_html(str(rap_box))[0]
# check
rap_table

,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,OG Anunoby,30:26,4,8,.500,3,5,.600,0,0,...,0,1,1,4,0,3,4,0,11,-15
1,Pascal Siakam,30:10,5,10,.500,0,2,.000,6,8,...,0,2,2,8,1,0,2,1,16,-7
2,Scottie Barnes,30:00,5,11,.455,1,4,.250,3,4,...,0,8,8,4,3,0,5,1,14,-22
3,Dennis Schröder,27:16,8,13,.615,4,4,1.000,4,4,...,0,1,1,1,0,0,2,2,24,-11
4,Jakob Poeltl,22:55,5,7,.714,0,0,NaN,0,0,...,1,9,10,1,0,1,4,6,10,-13
5,Reserves,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
6,Gary Trent Jr.,20:37,1,6,.167,1,2,.500,0,0,...,0,0,0,1,0,0,0,1,3,-3
7,Malachi Flynn,18:51,2,4,.500,1,2,.500,1,2,...,0,3,3,1,1,1,1,1,6,-14
8,Chris Boucher,17:50,1,4,.250,1,3,.333,0,0,...,0,2,2,0,0,1,0,3,3,-12
9,Precious Achiuwa,14:58,2,4,.500,0,1,.000,1,2,...,2,4,6,4,1,0,3,3,5,-9


## Get Schedule

In [783]:
url_schedule = 'https://www.basketball-reference.com/teams/TOR/2024_games.html'

In [784]:
# data_schedule = requests.get(url_schedule)
# with open('Box_Scores/Schedule.html', 'w+') as f:
#     f.write(data_schedule.text)
    
# with open('Box_Scores/Schedule.html') as f:
#     page_schedule = f.read()

# soup_schedule = BeautifulSoup(page_schedule, "html.parser")


# schedule = soup_schedule.find_all(id='games')



In [785]:
url_schedule = 'https://www.basketball-reference.com/teams/TOR/2024_games.html'
# Send an HTTP request to the URL
response = requests.get(url_schedule)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup_schedule = BeautifulSoup(response.text, 'html.parser')

    # Now you can work with the parsed HTML, for example, print the title
    # print("Title:", soup_schedule.title.text)
    
    # Here, you can add more code to extract and manipulate data from the HTML
else:
    print(f"Error: Unable to fetch the URL. Status code: {response.status_code}")
schedule = soup_schedule.find_all(id='games')

In [786]:
schedule_table = pd.read_html(str(schedule))[0]

In [787]:
# Replace column names with ones that make sense
new_columns={"Unnamed: 3": 'DateTime','Start (ET)': 'Time',"Unnamed: 5":"Location", 'Unnamed: 7': 'Result', 'Unnamed: 8': 'Overtime?', 'Notes':'In-Season Tournament'}
schedule_table.rename(columns=new_columns, inplace = True)
# Note In Season Tornaments
schedule_table['In-Season Tournament'].replace('In-Season Tournament', True, inplace = True)
# Overtime
schedule_table['Overtime?'].replace('OT', True, inplace = True)

# Replace Nan Location values with home
schedule_table.fillna({'Location':'Home', 'In-Season Tournament': False }, inplace=True)
# Remove extra rows 
schedule_table.drop([20,41,62], inplace=True)


In [788]:
# Fill DateTime column using Date and Time
for index, row in schedule_table.iterrows():
    schedule_table.at[index, 'DateTime'] = todtfromdf(schedule_table, 'Date', 'Time', index)
    if schedule_table.at[index, 'Unnamed: 4'] == 'Box Score' and schedule_table.at[index, 'Overtime?']!= True:
        schedule_table.at[index, 'Overtime?'] = False
    if schedule_table.at[index, 'Location'] == '@':
        schedule_table.at[index, 'Location'] = nba_teams[schedule_table.at[index, 'Opponent']]

KeyError: ''

In [789]:
schedule_table.reset_index(inplace = True)
schedule_table.drop(columns ='index', inplace=True)

In [790]:
schedule_table.head(40)

,G,Date,Time,DateTime,Unnamed: 4,Location,Opponent,Result,Overtime?,Tm,Opp,W,L,Streak,In-Season Tournament
0,1,"Wed, Oct 25, 2023",7:30p,2023-10-25 19:30:00,Box Score,Home,Minnesota Timberwolves,W,False,97,94,1,0,W 1,False
1,2,"Fri, Oct 27, 2023",8:00p,2023-10-27 20:00:00,Box Score,CHI,Chicago Bulls,L,True,103,104,1,1,L 1,False
2,3,"Sat, Oct 28, 2023",7:30p,2023-10-28 19:30:00,Box Score,Home,Philadelphia 76ers,L,False,107,114,1,2,L 2,False
3,4,"Mon, Oct 30, 2023",7:30p,2023-10-30 19:30:00,Box Score,Home,Portland Trail Blazers,L,False,91,99,1,3,L 3,False
4,5,"Wed, Nov 1, 2023",7:00p,2023-11-01 19:00:00,Box Score,Home,Milwaukee Bucks,W,False,130,111,2,3,W 1,False
5,6,"Thu, Nov 2, 2023",7:00p,2023-11-02 19:00:00,Box Score,PHI,Philadelphia 76ers,L,False,99,114,2,4,L 1,False
6,7,"Sun, Nov 5, 2023",3:30p,2023-11-05 15:30:00,Box Score,SAS,San Antonio Spurs,W,True,123,116,3,4,W 1,False
7,8,"Wed, Nov 8, 2023",8:30p,2023-11-08 20:30:00,Box Score,DAL,Dallas Mavericks,W,False,127,116,4,4,W 2,False
8,9,"Sat, Nov 11, 2023",7:00p,2023-11-11 19:00:00,Box Score,BOS,Boston Celtics,L,False,94,117,4,5,L 1,False
9,10,"Mon, Nov 13, 2023",7:30p,2023-11-13 19:30:00,Box Score,Home,Washington Wizards,W,False,111,107,5,5,W 1,False


In [791]:
find_last_occurrence(schedule_table,'Unnamed: 4','Box Score')

20

In [792]:
print(recent_game_url(schedule_table, column='Unnamed: 4', value='Box Score'))

202312060TOR


In [733]:
nba_teams = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}


In [738]:
def month_num(month):
    m_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4, 'May':5,'Apr':6,'Jun':7,'Jul':8,'Aug':9,'Oct':10,'Nov':11,'Dec':12}
    return str(m_dict[month])

def todtfromdf (df, datec, timec, row_num):
    ''' df, str, str, int -> datetime
    from dataframe takes date/time w format 'Wed, Oct 25, 2023' and '7:30p' reformates it to 2023/10/25 7:30:00
    Turns reformatted string into datetime
    Moves Date time up 12 hours (turning it into pm)
    ''' 
    date_str = df.at[row_num,datec][-4:] + '/' # add year
    date_str += month_num(df.at[row_num,datec][5:8])+ '/' # add month
    if df.at[row_num,datec][9:11][-1] == ',': # if day is single digit, second instance will be ','
        date_str += df.at[row_num,datec][9:10] + ' ' # if day is single digit
    else:
        date_str += df.at[row_num,datec][9:11] + ' ' # if day is double digit
    time_str = df.at[row_num,timec].rsplit('p')[0]

    date_str += time_str + ':00' # add time

    date_format = '%Y/%m/%d %H:%M:%S' # tell datetime how string is going to be formatted
    date_obj = datetime.strptime(date_str, date_format) # reformat datetime
    date_obj = date_obj + timedelta(hours=12) # move forward 12 hours
    return date_obj

In [743]:
recent_game_url = 'https://www.basketball-reference.com/boxscores/YYYYMMDD0LOC.html'

def find_last_occurrence(df, column_name, value):
    """
    Find the index of the last occurrence of a value in a specific column of a Pandas DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame to search.
    - column_name (str): The name of the column to search.
    - value: The value to find.

    Returns:
    - Index of the last occurrence of the value in the specified column.
      Returns None if the value is not found.
    """
    try:
        last_occurrence_index = df[df[column_name] == value].index[-1]
        return last_occurrence_index
    except IndexError:
        # IndexError will be raised if the value is not found
        return 'IndexError will be raised if the value is not found'
    

def recent_game_url(df, column = 'Unnamed: 4', value= 'Box Score'):
    ''' (df, str, str) -> str
    Uses schedule df and builds a string YYYYMMDD0LOC based on the most recent game

    YYYY = Year
    MM = Month
    DD = Day
    0
    LOC = 3 letter code either Home TOR or @ filled by nba teams dictionary
    '''
    YYYYMMDD0LOC = ''
    recent_game_index = find_last_occurrence(df, column, value)
    YYYYMMDD0LOC += str(df.iloc[recent_game_index][3].year)
    YYYYMMDD0LOC += df.iloc[recent_game_index][3].strftime('%m')
    YYYYMMDD0LOC += df.iloc[recent_game_index][3].strftime('%d')
    YYYYMMDD0LOC += '0'
    if df.iloc[recent_game_index][5] == 'Home':
        YYYYMMDD0LOC += 'TOR'
    elif df.iloc[recent_game_index][5] == '@':
        YYYYMMDD0LOC += nba_teams[schedule_table.iloc[recent_game_index][6]]

    return YYYYMMDD0LOC

def game_url(df, url = 'https://www.basketball-reference.com/boxscores/'):
    url += recent_game_url(df)
    url += '.html'
    return url

def num_of_threes(df):
    # grab number of 3's taken via last row (team totals), column 6 (3P)
    return(df.iloc[-1][5])

In [698]:
schedule_table.iloc[12][3]

datetime.datetime(2023, 11, 19, 16, 0)

In [699]:
# check if now is after 3 hours post game starting
def is_game_finished(df, column = 'Unnamed: 4', value = 'Box Score'):
    ''' (df)->(bool)
    Checks if time datetime.now is after most recent game is finished
    This function assumes nba games last MAX 3.5 hours. Typical games are 2-2.5
    Time now must be within a certain interval to ensure the game has recently ended (13.5 hours after game finishes)
    '''
    recent_game_index = find_last_occurrence(df, column, value)
    # Assumes game is done after 3.5 hours
    post_game_time = schedule_table.iloc[recent_game_index][3] + timedelta(hours=3.5)
    return datetime.now() > post_game_time and datetime.now() < post_game_time + timedelta(hours=10)

In [700]:
game_url(schedule_table)

'https://www.basketball-reference.com/boxscores/202312080CHA.html'

In [701]:
# Send an HTTP request to the URL
response = requests.get(game_url(schedule_table))

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Now you can work with the parsed HTML, for example, print the title
    #print("Title:", soup.title.text)
    
    # Here, you can add more code to extract and manipulate data from the HTML
    
else:
    print(f"Error: Unable to fetch the URL. Status code: {response.status_code}")
# Remove unnecessary top row with name: th.over_header.center
soup.find('tr', class_= 'over_header').decompose()
# name of table class: box-TOR-game-basic, grab table
rap_box = soup.find_all(id='box-TOR-game-basic')
# turn table class into a dataframe
rap_table = pd.read_html(str(rap_box))[0]
# check 3s 
num_of_threes(rap_table)

Error: Unable to fetch the URL. Status code: 404


'14'